# VALIDATION NOTEBOOK
This notebook loads all the audit files in the table master.audit:

In [1]:
#!pip install psycopg2
import os
import pandas as pd
from sqlalchemy import create_engine

We set the scale factor that we are interested:

In [2]:
SF=5

Now we save all the audit files of Batch1 - historical data load into one pandas dataframe

In [3]:
# Define the directory where the CSV files are located
paths = [f'./src/dags/data/sf_{SF}/Batch1', f'./src/dags/data/sf_{SF}/Batch2', f'./src/dags/data/sf_{SF}/Batch3']
dataframes = []

# Loop through all files in the directory
for batch_path in paths:
  for filename in os.listdir(batch_path):
      if filename.endswith('_audit.csv'):

          file_path = os.path.join(batch_path, filename)
          
          df = pd.read_csv(file_path)
        
          dataframes.append(df)

# Concatenate all dataframes into one large dataframe
final_df = pd.concat(dataframes, ignore_index=True)
final_df.columns = ['dataset', 'batchid', 'date', 'attribute', 'value', 'dvalue']
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

Now we set up the connection to the database and set up the first 3 parameters
This script fills up the `master.audit` table with the previous calculated dataframe:

In [4]:
user = 'postgres'
password = 'postgres'
port = '5432'

host = 'localhost'
database = f'sf_{SF}'

# Create a connection string for SQLAlchemy
conn_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Create a SQLAlchemy engine
engine = create_engine(conn_string)

table_name = 'master.audit' 

final_df.to_sql('audit', engine, schema='master', if_exists='replace', index=False)

print("DataFrame has been copied to PostgreSQL table 'master.audit'.")

DataFrame has been copied to PostgreSQL table 'master.audit'.


If we want to execute some of the validation tests - we just run one of the followings in the cmd (works for windows)

In [5]:
# Define the command to be executed
command = f'psql -h {host} -p {port} -U {user} -d {database} -f validation/tpcdi_audit.sql'
print(command)
command = f'psql -h {host} -p {port} -U {user} -d {database} -f validation/tpcdi_validation.sql'
print(command)
command = f'psql -h {host} -p {port} -U {user} -d {database} -f validation/tpcdi_visibility_1.sql'
print(command)
command = f'psql -h {host} -p {port} -U {user} -d {database} -f validation/tpcdi_visibility_2.sql'
print(command)

psql -h localhost -p 5432 -U postgres -d sf_5 -f validation/tpcdi_audit.sql
psql -h localhost -p 5432 -U postgres -d sf_5 -f validation/tpcdi_validation.sql
psql -h localhost -p 5432 -U postgres -d sf_5 -f validation/tpcdi_visibility_1.sql
psql -h localhost -p 5432 -U postgres -d sf_5 -f validation/tpcdi_visibility_2.sql
